**Original Polya-Gamma Optimization-based EM Model**

In [ ]:
import numpy as np
from tqdm import tqdm

class PolyaGammaEM:
    def __init__(self, num_items, num_workers, max_iter=500, epsilon=1e-4):
        self.num_items = num_items
        self.num_workers = num_workers
        self.max_iter = max_iter
        self.epsilon = epsilon

        # Initialize parameters with identifiability constraints
        self.r = np.random.randn(num_items)
        self.r -= self.r.mean()  # Zero-mean initialization
        self.beta = np.random.uniform(0.1, 0.9, num_workers)  # Avoid 0 initialization

    def fit(self, comparisons):
        prev_r = np.copy(self.r)
        prev_beta = np.copy(self.beta)

        for _ in tqdm(range(self.max_iter)):
            # E-step: Compute Polya-Gamma expectations
            e_omega = self._compute_pg_expectations(comparisons)

            # M-step: Update parameters
            self._update_rewards(comparisons, e_omega)
            self._update_competencies(comparisons, e_omega)

            # Enforce identifiability constraints
            self.r -= self.r.mean()
            self.beta = np.clip(self.beta, 1e-10, 1-1e-10)

            # Check convergence
            if self._check_convergence(prev_r, prev_beta):
                break
            prev_r = np.copy(self.r)
            prev_beta = np.copy(self.beta)

        return self.r, self.beta

    def _compute_pg_expectations(self, comparisons):
        e_omega = []
        for w, l, k in comparisons:
            x = self.beta[k] * (self.r[w] - self.r[l])
            if np.abs(x) < 1e-8:
                # Taylor expansion for numerical stability
                ew = 0.25 - (x**2)/48.0
            else:
                ew = np.tanh(x/2) / (2*x)
            e_omega.append(ew)
        return np.array(e_omega)

    def _update_rewards(self, comparisons, e_omega):
        numerator = np.zeros(self.num_items)
        denominator = np.zeros(self.num_items)

        for idx, (w, l, k) in enumerate(comparisons):
            beta_k = self.beta[k]
            ew = e_omega[idx]

            # Update for winning item
            numerator[w] += 0.5 * ew  # beta_k cancels out (see derivation)
            denominator[w] += ew * beta_k**2

            # Update for losing item
            numerator[l] -= 0.5 * ew
            denominator[l] += ew * beta_k**2

        # Avoid division by zero
        denominator = np.clip(denominator, 1e-10, None)
        self.r = numerator / denominator

    def _update_competencies(self, comparisons, e_omega):
        numerator = np.zeros(self.num_workers)
        denominator = np.zeros(self.num_workers)

        for idx, (w, l, k) in enumerate(comparisons):
            beta_k_old = self.beta[k]
            delta = self.r[w] - self.r[l]
            ew = e_omega[idx]

            # Compute update components
            factor = 0.5 / (beta_k_old + 1e-10)
            numerator[k] += ew * factor * delta
            denominator[k] += ew * delta**2

        # Apply updates only to workers with comparisons
        for k in range(self.num_workers):
            if denominator[k] > 1e-10:
                self.beta[k] = numerator[k] / denominator[k]

    def _check_convergence(self, prev_r, prev_beta):
        r_diff = np.linalg.norm(self.r - prev_r)
        beta_diff = np.linalg.norm(self.beta - prev_beta)
        return r_diff < self.epsilon and beta_diff < self.epsilon